# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

#read all csv files in the event_data folder 
file_path_list = glob.glob(os.path.join(filepath,'*.csv'))
#print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:
# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line)

# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

headers = ['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId']
with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(headers)
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Appache Cassandra part

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS cassandra_project_keyspace 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('cassandra_project_keyspace')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### <u>All 3 tables will be created in the next 3 cells </u>


### Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
### The music_history table is designed based on the wanted query based on conditions sessionId and itemInSession, so they both were chosen as partition keys. This way the tuple sessionId and itemInSession will be always unique.

In [8]:
# First drop the table in case it exists
try:
    rows = session.execute("drop table if exists music_history")
except Exception as e:
    print(e)
    
# Then create the table
try:
    rows = session.execute("""create table if not exists music_history (sessionId int,
    itemInSession int, artist text, song text, length float, primary key (sessionId, itemInSession))""")
except Exception as e:
    print(e)

### Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

### The user table is designed so that the partition keys are userId and sessionid based on the wanted query since they are shown as the partition keys,  and since the song should be sorted by itemInSession, then itemInSession was created as the clustering key. this way the tuple userId, sessionId and itemInSession will be always unique

In [9]:
# First drop the table in case it exists
try:
    rows = session.execute("drop table if exists user_history_sessions")
except Exception as e:
    print(e)
    
# Then create the table
try:
    rows = session.execute("""create table if not exists user_history_sessions (userId int,
    sessionId int, itemInSession int, firstName text, lastName text,
    song text, artist text, primary key ((userId, sessionId), itemInSession))""")
except Exception as e:
    print(e)

### Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

### The song table was created based on the query to select items based on the song name, and who listented to it (user), so song was chosen as the partition key and userId was chosen as the clustering key. In this table only song and userId tuple will be unique, as the history of the usage of each user is not desired, only one occurence from his usage.

In [10]:
# First drop the table in case it exists
try:
    rows = session.execute("drop table if exists song_usage_history")
except Exception as e:
    print(e)
    
# Then create the table
try:
    rows = session.execute("""create table if not exists song_usage_history (song text, 
    userId int, firstName text, lastName text, primary key (song, userId))""")
except Exception as e:
    print(e)

#### <u>In the next cell, all corresponding values will be inserted to each table</u>

In [11]:
# Go through the csv file line by line and insert the corresponding values to each table.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Assign the INSERT statements into the `query` variable for music_history table
        query_music = """insert into music_history (sessionId, itemInSession, artist, song, length)"""
        query_music = query_music + " values (%s, %s, %s, %s, %s) "
        # Assign which column element should be assigned for each column in the INSERT statement.
        query_music_values = (int(line[headers.index('sessionId')]), int(line[headers.index('itemInSession')]),\
                                line[headers.index('artist')], line[headers.index('song')],\
                                float(line[headers.index('length')]))
        session.execute(query_music, query_music_values)
        
        # Assign the INSERT statements into the `query` variable for user table
        query_user = """insert into user_history_sessions (userId, sessionId, itemInSession,
        firstName, lastName, song, artist)"""
        query_user = query_user + " values (%s, %s, %s, %s, %s, %s, %s) "
        # Assign which column element should be assigned for each column in the INSERT statement.
        query_user_values = (int(line[headers.index('userId')]), int(line[headers.index('sessionId')]),\
                                int(line[headers.index('itemInSession')]), line[headers.index('firstName')],\
                                line[headers.index('lastName')], line[headers.index('song')],\
                            line[headers.index('artist')])
        session.execute(query_user, query_user_values)
        
        # Assign the INSERT statements into the `query` variable for song table
        query_song = """insert into song_usage_history (userId, firstName, lastName, song)"""
        query_song = query_song + " values (%s, %s, %s, %s) "
        # Assign which column element should be assigned for each column in the INSERT statement.
        query_song_values = (int(line[headers.index('userId')]), line[headers.index('firstName')],\
                                line[headers.index('lastName')], line[headers.index('song')])
        session.execute(query_song, query_song_values)

#### SELECT to verify that the data have been inserted into each table

#### First, the music table

In [12]:
# A simple function to query the music table
def query_music(session_id_input, item_in_session_input):
    # The music table quered by selecting the desired columns and quering by the primary keys sessionId and itemInSession
    # as primary keys conditions, in this query artist, song and song length are needed so they were selected
    
    select_query_music = """select artist, song, length from music_history where\
    sessionId = {} and itemInSession = {}""".format(session_id_input, item_in_session_input)

    try:
        rows = session.execute(select_query_music)
    except Exception as e:
        print(e)

    if not rows:
        print("The query result is empty")
    else:
        music_query_list = []
        for row in rows:
            music_query_list.append(row)
        df_music_query = pd.DataFrame(music_query_list)
        display(df_music_query)

In [13]:
# Running queris with certain inputs
print("-Give me the artist, song title and song's length in the music app history that was heard during\
 sessionId = 338, and itemInSession  = 4")
query_music(338,4)

-Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession  = 4


,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


##### So there is one row in the table satisfying this condition (sessionId = 338 and itemInSession = 4)

#### Second, the user_history_sessions table

In [14]:
# A simple function to query the user table
def query_user(user_id_input, session_id_input):
    # The user_history_sessions table quered by selecting the desired columns and quering by the primary keys userId and sessionId as primary
    # keys conditions, in this query artist, song and user firstName and lastName are needed so they were selected,
    # As itemInSession is defined as a clustering column, song will be sorted according to it.
    
    select_query_user = """select artist, song, itemInSession, firstName, lastName from user_history_sessions where\
    userId = {} and sessionId = {}""".format(user_id_input, session_id_input)

    try:
        rows = session.execute(select_query_user)
    except Exception as e:
        print(e)

    if not rows:
        print("The query result is empty")
    else:
        user_query_list = []
        for row in rows:
            user_query_list.append(row)
        df_user_query = pd.DataFrame(user_query_list)
        display(df_user_query)

In [15]:
## Running queris with certain inputs
print("- Give me only the following: name of artist, song (sorted by itemInSession) and\
user (first and last name) for userid = 10, sessionid = 182")
query_user(10,182)


- Give me only the following: name of artist, song (sorted by itemInSession) anduser (first and last name) for userid = 10, sessionid = 182


,artist,song,iteminsession,firstname,lastname
0,Down To The Bone,Keep On Keepin' On,0,Sylvie,Cruz
1,Three Drives,Greece 2000,1,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,2,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,3,Sylvie,Cruz


##### Great, 4 rows as a results, and they are sorted by itemInSession as requested
                    

#### Third and last, the song_usage_history table

In [16]:
# A simple function to query the song table
def query_song(song_input):
    # The song_usage_history table quered by selecting the desired columns and quering by the primary key which is song 
    # in this query user firstName and lastName are needed so they were selected,
    # As userId is defined as a clustering column, songs will be sorted according to it.
    select_query_song = """select song, userId, firstName, lastName from song_usage_history where song = '{}'\
    """.format(song_input)

    try:
        rows = session.execute(select_query_song)
    except Exception as e:
        print(e)

    if not rows:
        print("The query result is empty")
    else:
        song_query_list = []
        for row in rows:
            song_query_list.append(row)
        df_song_query = pd.DataFrame(song_query_list)
        display(df_song_query)

In [17]:
# Running queris with certain inputs
print("- Give me every user name (first and last) in my music app history who listened to\
the song 'All Hands Against His Own'")
query_song('All Hands Against His Own')

- Give me every user name (first and last) in my music app history who listened tothe song 'All Hands Against His Own'


,song,userid,firstname,lastname
0,All Hands Against His Own,29,Jacqueline,Lynch
1,All Hands Against His Own,80,Tegan,Levine
2,All Hands Against His Own,95,Sara,Johnson


#### So now we insured that the 3 tables are populated

### Drop the tables before closing out the sessions

In [18]:
# Drop the tables before closing out the sessions

# drop music_history
try:
    rows = session.execute("drop table if exists music_history")
except Exception as e:
    print(e)
    
    
# drop song_usage_history table
try:
    rows = session.execute("drop table if exists song_usage_history")
except Exception as e:
    print(e)
    
# drop user_history_sessions table
try:
    rows = session.execute("drop table if exists user_history_sessions")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()